In [1]:
import pandas as pd

In [5]:
INDEX = "MS8"

#Carico tutti i dati
prices = pd.read_csv( f"./data{INDEX}/prices_W.csv" )
shorts = pd.read_csv( f"./data{INDEX}/short_interest_W.csv" )
trends = pd.read_csv( f"./data{INDEX}/trends_W.csv" )

In [6]:
prices['date'] = pd.to_datetime(prices['date'])
shorts['date'] = pd.to_datetime(shorts['date'])
trends['date'] = pd.to_datetime(trends['date'])

dates = pd.to_datetime(shorts['date'].unique())
dates = pd.Series(dates).sort_values().unique()

In [7]:
#modifico prices
prices = prices.sort_values(['ticker', 'date'])
prices_bi = pd.DataFrame()

for t in prices['ticker'].unique():
    sub = prices[prices['ticker'] == t]
    merged = pd.merge_asof(
        pd.DataFrame({'date': dates}),
        sub[['date', 'close']],
        on='date',
        direction='backward'
    )
    merged['ticker'] = t
    prices_bi = pd.concat([prices_bi, merged], ignore_index=True)

In [14]:
#modifico trends
trends_long = trends.melt(id_vars='date', var_name='ticker', value_name='trend')
trends_long = trends_long.sort_values(['ticker', 'date'])

trends_bi = pd.DataFrame()
for t in trends_long['ticker'].unique():
    sub = trends_long[trends_long['ticker'] == t]
    merged = pd.merge_asof(
        pd.DataFrame({'date': dates}),
        sub[['date', 'trend']],
        on='date',
        direction='backward'
    )
    merged['ticker'] = t
    trends_bi = pd.concat([trends_bi, merged], ignore_index=True)

In [19]:
df_merged = (
    shorts
    .merge(prices_bi, on=['date', 'ticker'], how='left')
    .merge(trends_bi, on=['date', 'ticker'], how='left')
)
df_merged = df_merged.dropna()

In [20]:
df_merged

,date,ticker,d2c,shorts,volume,close,trend
57,2020-06-30,AMC,2.82,17737420,6280921,41.7829,4.0
58,2020-07-15,AMC,4.40,16665023,3785508,45.9812,3.0
59,2020-07-31,AMC,5.78,17601239,3043253,40.3835,4.0
60,2020-08-14,AMC,2.28,16950844,7424764,55.3773,9.0
61,2020-08-31,AMC,1.94,17759866,9151188,62.9742,9.0
...,...,...,...,...,...,...,...
1241,2025-03-31,NOK,2.51,41961248,16740154,5.1630,3.0
1242,2025-04-15,NOK,1.50,35544079,23695078,4.9152,3.0
1243,2025-04-30,NOK,1.30,30226330,23310810,4.9450,3.0
1244,2025-05-15,NOK,1.83,30806409,16866181,5.0900,3.0


In [21]:
df_merged.to_csv(f"./data{INDEX}/merged_data.csv", index=False)